# Segmenting and Clustering Neighborhood in Toronto 

## Data Preparation 

In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [42]:
# Get requests 
requests = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
neighborhood = BeautifulSoup(requests.text) 
tables = neighborhood.find_all("table")

# Convert to pandas table 
table = []
for row in tables[0].find_all("tr"): 
    row_data = [] 
    
    for cell in row.find_all("td"): 
        row_data.append(cell.text)
    
    if not row_data == []: 
        table.append(row_data)

data = pd.DataFrame(table, columns = ["PostalCode", "Borough", "Neighborhood"])

# Remove \n 
data = data.replace("\n","", regex=True)

# Remove observations where Borough = Not assigned
data = data[data.Borough != "Not assigned"]
data

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [43]:
print(f"- There are {data.shape[0]} rows in the dataframe")

- There are 103 rows in the dataframe


In [45]:
# Add coordinates 
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
data = pd.merge(data, coordinates, left_on="PostalCode", right_on="Postal Code", how="left").drop(columns=["Postal Code"])

In [48]:
data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [49]:
print('- The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data['Borough'].unique()),
        data.shape[0]
    )
)

- The dataframe has 10 boroughs and 103 neighborhoods.


## Segmenting 


## Clustering